<a href="https://colab.research.google.com/github/phumipatc/CU_Submissions/blob/master/AI/Sound_to_Dementia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dataset: DementiaBank**
https://dementia.talkbank.org/


English Pitt Corpus: Cookie theft task
* https://dementia.talkbank.org/access/English/Pitt.html
* Dementia vs control



Preparing environment

In [1]:
%pip install openl3
%pip install pandas
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def tryImportColab():
  try:
    import google.colab
    return True
  except ImportError:
    return False

runningInColab = tryImportColab()
runningInColab

True

In [3]:
if(runningInColab):
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# **Audio Embedding**
# OpenL3
* https://openl3.readthedocs.io/en/latest/tutorial.html
* http://www.justinsalamon.com/uploads/4/3/9/4/4394963/cramer_looklistenlearnmore_icassp_2019.pdf

# AudioSet
* https://github.com/tensorflow/models/tree/master/research/audioset
* Use vggish
* Or, https://tfhub.dev/google/vggish/1

# Other embedding models
* https://tfhub.dev/s?module-type=audio-embedding


In [4]:
%pwd

'/content'

In [5]:
import pandas as pd

address_sample_csv_path = 'sound-dementia-data/ADReSS-M-sample-gr/'
if runningInColab:
  address_sample_csv_path = 'drive/MyDrive/' + address_sample_csv_path

# Get dataFrame
address_sample_original_df = pd.read_csv(address_sample_csv_path + "sample-gr-groundtruth.csv")
address_sample_clean_df = address_sample_original_df.dropna().drop_duplicates()

# Cleaning Process
## In gender col, change "Female" to 0 and "Male" to 1
address_sample_clean_df['gender'] = address_sample_clean_df['gender'].apply(lambda x: 0 if x == "Female" else 1)
## In dx column, change "Control" to 0 and "ProbableAD" to 1
address_sample_clean_df['dx'] = address_sample_clean_df['dx'].apply(lambda x: 0 if x == "Control" else 1)

address_sample_clean_df.corr()

<ipython-input-5-e442b690c847>:17: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  address_sample_clean_df.corr()


,age,gender,educ,dx,mmse
age,1.000000,-2.211016e-01,0.111367,6.747378e-01,-0.507624
gender,-0.221102,1.000000e+00,0.587643,6.409876e-17,0.137361
educ,0.111367,5.876425e-01,1.000000,-4.240945e-02,0.383415
dx,0.674738,6.409876e-17,-0.042409,1.000000e+00,-0.793052
mmse,-0.507624,1.373606e-01,0.383415,-7.930516e-01,1.000000


In [9]:
import openl3
import os
import soundfile as sf

address_sample_path = 'sound-dementia-data/ADReSS-M-sample-gr/sample-gr/'
if runningInColab:
  address_sample_path = 'drive/MyDrive/' + address_sample_path
address_sample_list = []
for fName in os.listdir(address_sample_path):
  address_sample_list.append(address_sample_path + fName)
address_sample_list

['drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs097.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs046.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs047.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs009.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs054.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs002.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs087.wav',
 'drive/MyDrive/sound-dementia-data/ADReSS-M-sample-gr/sample-gr/madrs081.wav']

In [10]:
result = []
for sample in address_sample_list:
  audio, sr = sf.read(sample)
  embedding, timestamps = openl3.get_audio_embedding(audio, sr)
  result.append([embedding, timestamps])

10/10 [==============================] - 3s 281ms/step


In [11]:
result

[[array([[ 2.430838  ,  1.1149532 ,  1.8192567 , ..., -0.40260366,
           2.2451284 ,  1.161053  ],
         [ 2.430838  ,  1.1149532 ,  1.8192567 , ..., -0.0993605 ,
           2.1806672 ,  1.1700141 ],
         [ 3.1156824 ,  1.1149518 ,  2.544499  , ...,  0.21734096,
           2.1746001 ,  1.100827  ],
         ...,
         [ 1.8850679 ,  1.0349948 ,  2.3077831 , ..., -0.08238058,
           2.2199914 ,  1.214437  ],
         [ 1.8134855 ,  0.83887595,  1.6729833 , ...,  0.04376705,
           2.2287323 ,  1.1768765 ],
         [ 1.9946804 ,  0.7640471 ,  1.6467764 , ...,  0.01670596,
           3.4593952 ,  1.2039174 ]], dtype=float32),
  array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
          1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
          2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
          3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
          4.4,  4.5,  4.6,  4.7,  4.8,  4.9, 

# **Classification**
# Classics
* https://scikit-learn.org/stable/supervised_learning.html
* Logistic regression, Support Vector Classification, Decision Tree, Random Forest, Neural Net, AdaBoost, Naïve Bayes
* https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

# Classification heads
* https://www.isca-speech.org/archive/pdfs/interspeech_2021/gauder21_interspeech.pdf
* Neural networks - Conv1D (k=1), Conv1D (k=3), Global. Average
* https://www.isca-speech.org/archive/pdfs/interspeech_2021/wang21ca_interspeech.pdf-Neural networks - Conv - Conv1D - Softmax
* Others
* https://www.tensorflow.org/tutorials/images/transfer_learning#add_a_classification_head

**Classic - Logistic Regressing**

In [ ]:
import sklearn.linear_model